In [61]:
import pandas as pd
import numpy as np

In [95]:
df = pd.read_csv("Year Order Item Data.csv", parse_dates=['date'])

df.to_dict(orient="records")

[{'restaurant_name': 'Makers of Milkshakes - Prozone Mall',
  'invoice_no': 1,
  'date': Timestamp('2024-04-01 11:32:59'),
  'payment_type': 'Cash',
  'order_type': 'Pick Up',
  'status': 'Success',
  'area': 'Parcel',
  'virtual_brand_name': nan,
  'brand_grouping': 'Offline Menu',
  'assign_to': nan,
  'customer_phone': 8390406775.0,
  'customer_name': 'slma saphir',
  'customer_address': nan,
  'persons': nan,
  'order_cancel_reason': nan,
  'my_amount': 344.23,
  'total_tax': 30.78,
  'discount': 0.0,
  'delivery_charge': 0,
  'container_charge': 0,
  'service_charge': 0,
  'additional_charge': 0,
  'waived_off': 0,
  'round_off': -0.01,
  'total': 375,
  'item_name': 'Strawberry Shake (Milkshake)',
  'category_name': 'Classic Shakes',
  'sap_code': nan,
  'item_price': 160.64,
  'item_quantity': 1,
  'item_total': 160.64},
 {'restaurant_name': 'Makers of Milkshakes - Prozone Mall',
  'invoice_no': 1,
  'date': Timestamp('2024-04-01 11:32:59'),
  'payment_type': 'Cash',
  'order_ty

In [63]:
df.dropna(subset=['date', 'item_name', 'item_quantity', 'item_total'], inplace=True)
df = df[df['status'].str.lower() == 'success']

numeric_cols = ['item_quantity', 'item_price', 'item_total', 'discount', 'waived_off']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df['discount'].fillna(0, inplace=True)
df['waived_off'].fillna(0, inplace=True)

# Dropping rows with NaN after coercion
df.dropna(subset=['item_quantity', 'item_price', 'item_total'], inplace=True)

# Filter for invalid sales data
df = df[(df['item_quantity'] > 0) & (df['item_total'] > 0)]

C:\Users\Rikhil Nellimarla\AppData\Local\Temp\ipykernel_7084\271279476.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['discount'].fillna(0, inplace=True)
C:\Users\Rikhil Nellimarla\AppData\Local\Temp\ipykernel_7084\271279476.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

In [ ]:
df['net_sales'] = df['item_total'] - df['discount'] - df['waived_off']
df['YearMonth'] = df['date'].dt.to_period('M')
df['Month'] = df['date'].dt.strftime('%b')  # Jan, Feb...

## Yearly

### Yearly: Top Selling

In [65]:
top_items_yearly = (
    df.groupby('item_name')['item_quantity']
    .sum()
    .reset_index()
    .sort_values(by='item_quantity', ascending=False)
)

In [ ]:
top_items_yearly.to_csv("../results/product_analysis/yearly/Yearly_Top_Selling_Items.csv", index=False)

### Yearly: Product Performance

In [67]:
yearly_kpis = (
    df.groupby('item_name').agg(
        total_units_sold=('item_quantity', 'sum'),
        total_net_sales=('net_sales', 'sum'),
        average_selling_price=('item_price', 'mean'),
        orders_count=('invoice_no', 'nunique')
    ).reset_index()
)

In [68]:
yearly_kpis['weighted_avg_price'] = yearly_kpis['total_net_sales'] / yearly_kpis['total_units_sold']
yearly_kpis.sort_values(by='total_units_sold', ascending=False, inplace=True)

In [ ]:
yearly_kpis.to_csv("../results/product_analysis/yearly/Yearly_Product_Performance.csv", index=False)

## Monthly

### Monthly Top Selling

In [70]:
monthly_pivot = (
    df.groupby(['item_name', 'Month'])['item_quantity']
    .sum()
    .unstack(fill_value=0)
)

monthly_pivot['Total'] = monthly_pivot.sum(axis=1)
monthly_pivot = monthly_pivot.sort_values(by='Total', ascending=False)
monthly_pivot.reset_index(inplace=True)

In [ ]:
monthly_pivot.to_csv("../results/product_analysis/monthly/Monthly_Top_Selling_Items.csv", index=False)

### Monthly: Product Performance

In [72]:
monthly_kpis = (
    df.groupby(['YearMonth', 'item_name']).agg(
        total_units_sold=('item_quantity', 'sum'),
        total_net_sales=('net_sales', 'sum'),
        average_selling_price=('item_price', 'mean'),
    ).reset_index()
)

monthly_kpis['YearMonth'] = monthly_kpis['YearMonth'].astype(str)

In [ ]:
monthly_kpis.to_csv("../results/product_analysis/monthly/Monthly_Product_Performance.csv", index=False)

### Daily

In [83]:
df['Order_Date'] = df['date'].dt.date
df['DayOfWeek'] = df['date'].dt.day_name()

In [ ]:
daily_kpis = (
    df.groupby(['Order_Date', 'DayOfWeek', 'item_name']).agg(
        total_units_sold=('item_quantity', 'sum'),
        total_net_sales=('net_sales', 'sum'),
        average_selling_price=('item_price', 'mean'),
        orders_count=('invoice_no', 'nunique')
    ).reset_index()
)

daily_kpis['weighted_avg_price'] = (
    daily_kpis['total_net_sales'] / daily_kpis['total_units_sold']
)

daily_kpis.to_csv("../results/product_analysis/daily/Daily_Product_Performance.csv", index=False)


In [ ]:
daily_kpis.to_csv("../results/product_analysis/daily/Daily_Product_Performance.csv", index=False)

In [ ]:
daily_pivot = (
    df.groupby(['DayOfWeek', 'item_name'])['item_quantity']
    .sum()
    .unstack(fill_value=0)
    .reindex([
        'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
    ])
)

daily_pivot.to_csv("../results/product_analysis/daily/Daily_Item_Sales_Pivot.csv")


In [ ]:
avg_day_kpi = (
    df.groupby('DayOfWeek').agg(
        avg_units_sold=('item_quantity', 'mean'),
        avg_net_sales=('net_sales', 'mean'),
        avg_orders=('invoice_no', 'nunique')
    ).reindex([
        'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
    ]).reset_index()
)

avg_day_kpi.to_csv("../results/product_analysis/daily/Average_Performance_By_DayOfWeek.csv", index=False)

### Hourly

In [86]:
df['Hour'] = df['date'].dt.hour

In [ ]:
# aggregated table
hourly_kpis = (
    df.groupby([df['date'].dt.date, 'Hour', 'item_name']).agg(
        total_units_sold=('item_quantity', 'sum'),
        total_net_sales=('net_sales', 'sum'),
        average_selling_price=('item_price', 'mean'),
        orders_count=('invoice_no', 'nunique')
    ).reset_index()
)

hourly_kpis['weighted_avg_price'] = hourly_kpis['total_net_sales'] / hourly_kpis['total_units_sold']
hourly_kpis.rename(columns={'date': 'Order_Date'}, inplace=True)
hourly_kpis.to_csv("../results/product_analysis/hourly/Hourly_Product_Performance.csv", index=False)


In [ ]:
# Pivot Table
pivot = df.groupby(['Hour', 'item_name'])['item_quantity'].sum().unstack(fill_value=0)
pivot.to_csv("../results/product_analysis/hourly/Hourly_Item_Sales_Pivot.csv")

In [ ]:
# average hour performance
hour_avg_kpi = (
    df.groupby('Hour').agg(
        avg_units_sold=('item_quantity', 'mean'),
        avg_net_sales=('net_sales', 'mean'),
        avg_orders=('invoice_no', 'nunique')
    ).reset_index()
)

hour_avg_kpi.to_csv("../results/product_analysis/hourly/Average_Performance_By_Hour.csv", index=False)